# Problem 3 Sample Code

This sample code is meant as a guide on how to use PyTorch and how to use the relevant model layers. This not a guide on how to design a network and the network in this example is intentionally designed to have poor performace.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from torch.utils.data import TensorDataset, Dataset, DataLoader
import seaborn as sns
from sklearn.model_selection import train_test_split
from scipy import stats

In [6]:
df = pd.read_csv('out.csv')
df = df.drop(columns= "uid")
df = df.replace('',np.nan)
df = df.dropna(axis="rows", how="any")

df[(np.abs(stats.zscore(df)) < 3).any(axis=1)]
# print("===dataframe=== \n {}".format(df))
# print ("df after drop \n{}".format(df))
# df['split'] = np.random.randn(df.shape[0], 1)
# msk = np.random.rand(len(df)) <= 0.7

# train = df[msk]
# train.drop(['split'], axis=1, inplace=True)
# test = df[~msk]
# test.drop(['split'], axis=1, inplace=True)
X = df.copy()
X = X.drop(columns=["label"])


# print(" - pre X \n {} - ".format(X))

for col in X:
    X[col] = [(element - np.mean(X[col])) / np.std(X[col]) for element in X[col]]

X = np.array(X)

# def normalize(x):
#     x = np.asarray(x)
#     return (x - x.min()) / (np.ptp(x))
# X_normed = []
# for i in range(0, len(X)):
#     X_normed.append(X[i])
# # X = (X - X.mean())/(X.std())
# print(" - NORMALIZED  X \n {} - ".format(X_normed))

y = np.array(df["label"])

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)
# print(" pre TRAIN X \n {}".format(train_X))

train_X = torch.tensor(train_X, dtype=torch.float32)
# print("TENSOR  X \n {}".format(train_X))
train_y = torch.tensor(train_y, dtype=torch.float32)
# 80/20 x values as floats
test_X = torch.tensor(test_X, dtype=torch.float32)
test_y = torch.tensor(test_y, dtype=torch.float32)

train_dataset = TensorDataset(train_X, train_y)
test_dataset = TensorDataset(test_X, test_y)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=10, shuffle=True) 

# print ("x \n{}".format(df))
# print ("y \n{}".format(df))


# x_train = torch.tensor(train[['chi','lin_reg','lin_reg_chi','stddev_step_speed']].copy().to_numpy(), dtype=torch.float32)
# y_train = torch.tensor(train['label'].copy().to_numpy(), dtype=torch.long)
# x_test = torch.tensor(train[['chi','lin_reg','lin_reg_chi','stddev_step_speed']].copy().to_numpy(), dtype=torch.float32)
# y_test = torch.tensor(train['label'].copy().to_numpy(), dtype=torch.long)
# train_dataset = TensorDataset(x_train, y_train)
# test_dataset = TensorDataset(x_test, y_test)

We can convert images to numpy arrays and plot them with matplotlib:

## Network Definition
Let's instantiate a model and take a look at the layers.

In [7]:
model = nn.Sequential(
    nn.Linear(4, 2),
    nn.ReLU(),
    nn.Linear(2, 1),
    nn.Sigmoid()
)

# def init_weights(m):
#     if isinstance(m, nn.Linear):
#         torch.nn.init.xavier_uniform(m.weight)
#         m.bias.data.fill_(0.01)

# model.apply(init_weights)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
loss_fn = nn.BCELoss()
# loss_fn = nn.CrossEntropyLoss()
torch.manual_seed(15552494823729223621)

## Training
We also choose an optimizer and a loss function.

We could write our training procedure manually and directly index the `Dataset` objects, but the `DataLoader` object conveniently creates an iterable for automatically creating random minibatches:

We now write our backpropagation loop, training for 10 epochs.

In [8]:
model.train()

for epoch in range(20):
    for batch_idx, (data, target) in enumerate(train_loader):
        # print("data \n{}".format(data))
        # Erase accumulated gradients

        optimizer.zero_grad()

        # Forward pass
        output = model(data)
        #print(" -- OPUTPUT -- \n{}".format(output))
        output = output.flatten()
        #print(" -- OUTPUT -- \n{}".format(output))
        # print(" -- TARGET -- \n{}".format(target))

        # Calculate loss
        loss = loss_fn(output, target)

        # Backward pass
        loss.backward()
        
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        torch.nn.utils.clip_grad_value_(model.parameters(), 10000)

        # Weight update
        optimizer.step()

    # Track loss each epoch
    print('Train Epoch: %d  Loss: %.4f' % (epoch + 1,  loss.item()))

Train Epoch: 1  Loss: 0.7158
Train Epoch: 2  Loss: 0.7180
Train Epoch: 3  Loss: 0.7653
Train Epoch: 4  Loss: 0.8494
Train Epoch: 5  Loss: 0.6746
Train Epoch: 6  Loss: 0.8875
Train Epoch: 7  Loss: 0.5773
Train Epoch: 8  Loss: 0.8226
Train Epoch: 9  Loss: 0.6216
Train Epoch: 10  Loss: 0.6863
Train Epoch: 11  Loss: 0.6617
Train Epoch: 12  Loss: 0.7768
Train Epoch: 13  Loss: 0.7753
Train Epoch: 14  Loss: 0.7247
Train Epoch: 15  Loss: 0.6396
Train Epoch: 16  Loss: 0.6847
Train Epoch: 17  Loss: 0.6783
Train Epoch: 18  Loss: 0.6161
Train Epoch: 19  Loss: 0.6986
Train Epoch: 20  Loss: 0.6812


## Testing
We can perform forward passes through the network without saving gradients.

In [9]:
# Putting layers like Dropout into evaluation mode
model.eval()

test_loss = 0
correct = 0

# Turning off automatic differentiation
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        test_loss += loss_fn(output, target).item()  # Sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max class score
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('Test set: Average loss: %.4f, Accuracy: %d/%d (%.4f)' %
      (test_loss, correct, len(test_loader.dataset),
       100. * correct / len(test_loader.dataset)))

ValueError: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])) is deprecated. Please ensure they have the same size.